Datasource: "Leafsnap: A Computer Vision System for Automatic Plant Species Identification,"  
Neeraj Kumar, Peter N. Belhumeur, Arijit Biswas, David W. Jacobs, W. John Kress, Ida C. Lopez, João V. B. Soares,  
Proceedings of the 12th European Conference on Computer Vision (ECCV),  
October 2012

In [1]:
tarurl = r'http://leafsnap.com/static/dataset/leafsnap-dataset.tar'

In [16]:
pwd

'C:\\Users\\User\\Documents\\GitHub\\datasets\\dataset\\images\\field'

In [2]:
%cd ../../datasets/

C:\Users\User\Documents\GitHub\datasets


In [ ]:
!curl -O $tarurl 

In [ ]:
!tar -xvf leafsnap-dataset.tar

## Imports

In [45]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import skimage
import os
import math
import random

In [18]:
%cd ./dataset/images/field/

[WinError 3] The system cannot find the path specified: './dataset/images/field/'
C:\Users\User\Documents\GitHub\datasets\dataset\images\field


In [19]:
tree_specs = os.popen('ls').read()
tree_specs = tree_specs.strip().split(sep='\n')

In [38]:
!mkdir test,train,vdate

A subdirectory or file test already exists.
Error occurred while processing: test.
A subdirectory or file train already exists.
Error occurred while processing: train.
A subdirectory or file vdate already exists.
Error occurred while processing: vdate.


In [ ]:
#move images into train, test, validate folders with subfolders for classes
test_ratio = .15
for spec in tree_specs:
    %cd ./$spec
    listing = os.popen('ls').read().strip().split(sep='\n')
    random.shuffle(listing) #inplace
    im_count = len(listing)
    test_size=val_size = math.ceil(test_ratio*im_count)
    train_size = im_count - (test_size + val_size)
    subfolders = ['train', 'test', 'vdate']
    for subfolder in subfolders:
        os.makedirs(os.path.join(os.path.dirname(os.getcwd()), subfolder, spec))
    for item in range(0, train_size):
        moved = listing.pop()
        !mv $moved ../train/$spec/$moved
    for item in range(0, test_size):
        moved = listing.pop()
        !mv $moved ../test/$spec/$moved
    for item in range(0, val_size):
        moved = listing.pop()
        !mv $moved ../vdate/$spec/$moved
    %cd ..
    !rmdir ./$spec

In [41]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)
vdate_datagen = ImageDataGenerator(rescale=1/255)

In [61]:
# Flow training images in batches of 128 using train_datagen generator
batch_size = 32
target_size = (256,256)
train_generator = train_datagen.flow_from_directory(
        './train/',  # This is the source directory for training images
        target_size=target_size, 
        batch_size=batch_size,
        # Specify the classes explicitly
        classes = tree_specs,
        class_mode='categorical')

Found 1303 images belonging to 48 classes.


In [62]:
test_generator = test_datagen.flow_from_directory(
        './test/',
        target_size=target_size,
        batch_size=batch_size,
        classes=tree_specs,
        class_mode='categorical')

Found 272 images belonging to 48 classes.


In [63]:
vdate_generator = vdate_datagen.flow_from_directory(
        './vdate/',
        target_size=target_size,
        batch_size=batch_size,
        classes=tree_specs,
        class_mode='categorical')

Found 272 images belonging to 48 classes.


# Modeling
### Sequential Model

In [96]:
sequential_mod = tf.keras.models.Sequential()
sequential_mod.add(Dense(64, activation='relu', input_shape=(256,256,3)))
sequential_mod.add(Dropout(.1))
sequential_mod.add(Dense(64, activation='relu'))
sequential_mod.add(Dropout(.1))
sequential_mod.add(Dense(48, activation='softmax'))

sequential_mod.summary()

sequential_mod.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(),
              metrics=['accuracy'])

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 256, 256, 64)      256       
_________________________________________________________________
dropout_32 (Dropout)         (None, 256, 256, 64)      0         
_________________________________________________________________
dense_56 (Dense)             (None, 256, 256, 64)      4160      
_________________________________________________________________
dropout_33 (Dropout)         (None, 256, 256, 64)      0         
_________________________________________________________________
dense_57 (Dense)             (None, 256, 256, 48)      3120      
Total params: 7,536
Trainable params: 7,536
Non-trainable params: 0
_________________________________________________________________


In [97]:
total_sample = train_generator.n

In [99]:
n_epochs = 10
history = sequential_mod.fit_generator(
        train_generator, 
        steps_per_epoch=int(total_sample/batch_size),  
        epochs=n_epochs,
        verbose=1)

Epoch 1/10


ValueError: A target array with shape (32, 48) was passed for an output of shape (None, 256, 256, 48) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.